In [104]:
pip install -r requirements.txt

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [105]:
import gitlab
import pandas as pd
import os
import time

In [ ]:
department = "vnpay"
host = 'http://localhost:10001'
private_token = ''
info_folder = os.path.join('info', department)

In [107]:
gl = gitlab.Gitlab(url=host, private_token=private_token)
gl.auth()

/tmp/ipykernel_42103/3625467838.py:2: UserWarning: The base URL in the server response differs from the user-provided base URL (http://localhost:10001 -> http://localhost:8929).
This is usually caused by a misconfigured base URL on your side or a misconfigured external_url on the server side, and can lead to broken pagination and unexpected behavior. If this is intentional, use `keep_base_url=True` when initializing the Gitlab instance to keep the user-provided base URL. (python-gitlab: /tmp/ipykernel_42103/3625467838.py:2)
  gl.auth()


In [108]:
username =gl.user.username

In [109]:
try:
    zgroup = gl.groups.create({'name': department, 'path': department})
except Exception as ex:
    print(ex)
group = gl.groups.get(department)
parent_id = group.id

400: Failed to save group {:path=>["has already been taken"]}


In [110]:
cfg_group_file = f'info/{department}/organization-export.csv'
df_group = pd.read_csv(cfg_group_file, index_col=0)
df_group.head()

,id,name,path,url
0,129816109,blc-onchain,blc-onchain,https://github.com/blc-onchain
1,188412566,ntkongdm,ntkongdm,https://github.com/ntkongdm


In [111]:
for index, group in df_group.iterrows():
    print(group['name'])
    group_name = group['name']
    try:
        output = gl.groups.create({
            'name': group_name,
            'path': group_name,
            'parent_id': parent_id,
        })
        print(f"Done create group {group_name}")
    except Exception as ex:
        print(f"Error while importing group {group_name}: {ex}")

blc-onchain
Error while importing group blc-onchain: 400: Failed to save group {:name=>["has already been taken"], :path=>["has already been taken"]}
ntkongdm
Error while importing group ntkongdm: 400: Failed to save group {:name=>["has already been taken"], :path=>["has already been taken"]}


In [112]:
cfg_project_file = f'info/{department}/repository-export.csv'
df_project = pd.read_csv(cfg_project_file, index_col=0)
df_project.head()

,id,name,name_with_namespace,folder,project_file,group,url
0,601876893,bot-chat-gpt,blc-onchain/bot-chat-gpt,info/vnpay/blc-onchain,info/vnpay/blc-onchain/bot-chat-gpt,blc-onchain,https://github.com/blc-onchain/bot-chat-gpt
1,888238368,temp,ntkongdm/temp,info/vnpay/ntkongdm,info/vnpay/ntkongdm/temp,ntkongdm,https://github.com/ntkongdm/temp


In [113]:
group = gl.groups.get(department)
projects = df_project.to_dict('records')

In [114]:
host_with_user = host.replace("//", f"//{username}:{private_token}@")

In [115]:
namespaces = gl.namespaces.list(get_all=True)
list_groups = []
for namespace in namespaces:
   list_groups.append({
         'id': namespace.id,
         # 'name': dgroup.name,
         # 'path': dgroup.path,
         # 'full_name': dgroup.full_name,
         'full_path': namespace.full_path,
         # 'parent_id': dgroup.parent_id,
         # 'created_at': dgroup.created_at,
         # 'organization_id': dgroup.organization_id,
    })
# print(list_groups)
list_groups

[{'id': 1, 'full_path': 'root'},
 {'id': 28, 'full_path': 'test'},
 {'id': 50, 'full_path': 'vnpay'},
 {'id': 51, 'full_path': 'vnpay/blc-onchain'},
 {'id': 52, 'full_path': 'vnpay/ntkongdm'}]

In [116]:
def get_fullpath(name, name_with_namespace):
    # print(name_with_namespace)
    parts = name_with_namespace.split('/')
    parts.pop()
    # print(parts)
    parts.insert(0, name)
    return '/'.join(str(x).strip() for x in parts) 
def get_parent(groups, project_path):
    for group in groups:
        if group['full_path'] == project_path: 
            return group['id']
    return None 

def found_project(project_name_with_namespace):
    try:
      return gl.projects.get(project_name_with_namespace)
    except:
      return None 

In [117]:
def create_new_project(project_name, parent_id):
    try:
        # Tạo repository mới
        new_project = gl.projects.create(
            name=project_name,
            namespace_id=parent_id,
            visibility='private',
        )
        return new_project
    except Exception as e:
        print(f"Error while creating project {project_name}: {e}")
        return None

In [118]:
import_status = []
for project in projects: 
    project_file = project['project_file']
    project_path = get_fullpath(department, project['name_with_namespace'])
    project_name = project['name']
    print(f"Start importing project: {project_name}, path: {project_path}")
    
    project_name_with_namespace = f"{project_path}/{project_name}"
    project = found_project(project_name_with_namespace)

    if project: 
        print(f"Project: {project_name_with_namespace} exists")
        import_status.append({
            'name': project_name,
            'name_with_namespace': project_name_with_namespace,
            'project_url': f"{host}/{project_name_with_namespace}",
            'ex': f"Project: {project_name_with_namespace} exists",
            'status': "exists"
        })
        continue 
    
    parent_id = get_parent(list_groups, project_path)
    if not parent_id: 
        parent_id = group.id  # Default in department group 
        print(f"Project: {project_name} not found parent id")
    
    try:
        # Tạo project mới trên GitLab hoặc hệ thống Git của bạn
        new_project = gl.projects.create({
            'name': project_name,
            'namespace_id': parent_id,
            'visibility': 'private'
        })
        print(f"Push {project_file} to {host}/{project_name_with_namespace}")
        
        # Lệnh push tất cả các nhánh và tags lên remote mới
        os.system(f"cd {project_file} && git push --all {host_with_user}/{project_name_with_namespace}")
        os.system(f"cd {project_file} && git push --tags {host_with_user}/{project_name_with_namespace}")
        import_status.append({
            'name': project_name,
            'name_with_namespace': project_name_with_namespace,
            'project_url': f"{host}/{project_name_with_namespace}",
            'ex': f'{None}',
            'status': "success"
        })
    except Exception as e:
        import_status.append({
            'name': project_name,
            'name_with_namespace': project_name_with_namespace,
            'project_url': f"{host}/{project_name_with_namespace}",
            'ex': f'{ex}',
            'status': "failed"
        })
        print(f"An unexpected error occurred while creating project {project_name}: {e}")
    
print("Done pushing all branches and tags to the new remote.")

Start importing project: bot-chat-gpt, path: vnpay/blc-onchain
Project: vnpay/blc-onchain/bot-chat-gpt exists
Start importing project: temp, path: vnpay/ntkongdm
Project: vnpay/ntkongdm/temp exists
Done pushing all branches and tags to the new remote.


In [119]:
import_status

[{'name': 'bot-chat-gpt',
  'name_with_namespace': 'vnpay/blc-onchain/bot-chat-gpt',
  'project_url': 'http://localhost:10001/vnpay/blc-onchain/bot-chat-gpt',
  'ex': 'Project: vnpay/blc-onchain/bot-chat-gpt exists',
  'status': 'exists'},
 {'name': 'temp',
  'name_with_namespace': 'vnpay/ntkongdm/temp',
  'project_url': 'http://localhost:10001/vnpay/ntkongdm/temp',
  'ex': 'Project: vnpay/ntkongdm/temp exists',
  'status': 'exists'}]

In [120]:
df = pd.DataFrame(import_status)
df

,name,name_with_namespace,project_url,ex,status
0,bot-chat-gpt,vnpay/blc-onchain/bot-chat-gpt,http://localhost:10001/vnpay/blc-onchain/bot-c...,Project: vnpay/blc-onchain/bot-chat-gpt exists,exists
1,temp,vnpay/ntkongdm/temp,http://localhost:10001/vnpay/ntkongdm/temp,Project: vnpay/ntkongdm/temp exists,exists
